In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import sys

sys.path.append("..")

In [ ]:
from constrerl.evaluate import (
    eval_submission_6_1_NER,
    eval_submission_6_3_ternary_tag_RE,
    eval_submission_6_4_ternary_mention_RE,
    eval_submission_6_2_binary_tag_RE,
)
from constrerl.erl_schema import convert_to_output, Article
import glob
from pathlib import Path
import json
import pandas as pd
from collections.abc import Callable, Awaitable


In [284]:
results_dir = "../data/results_dev"
ground_truth_file = "../data/annotations/dev/dev.json"
report_dir = Path("report")
results_dir = Path(results_dir)
ner_result_dir = Path("../data/results_ner_dev")
ground_truth_file = Path(ground_truth_file)
lbl_xtra = ":more"

top_results = 0

with open(ground_truth_file) as f:
    ground_truth = json.load(f)

In [285]:
eval_results: list[dict] = []

import re


def scoring_to_dict(
    f: str | Path, eval_f: Callable[[str | Path, dict], tuple[float]]
) -> dict:
    precision, recall, f1, micro_precision, micro_recall, micro_f1 = eval_f(
        f, ground_truth
    )
    return {
        "$P$": precision,
        "$R$": recall,
        "$F_1$": f1,
        "$P_{micro}$": micro_precision,
        "$R_{micro}$": micro_recall,
        "$F_{1,micro}$": micro_f1,
    }


def scoring_to_df(
    eval_f: Callable[[str | Path, dict], tuple[float]], res_dir=results_dir
) -> pd.DataFrame:
    eval_results: list[dict] = []
    merge_mode = "merge" in str(res_dir)
    further_mode = not (("old" in str(res_dir)) or ("merge" in str(res_dir)))
    print(f"Further mode: {further_mode}")
    for result_file in res_dir.glob("*.json"):
        set_mode = False
        result_file = Path(result_file)
        if "union" in result_file.name or "intersection" in result_file.name:
            set_mode = True
        set_op = "$\cup$" if "union" in result_file.name else "$\cap$"
        eval_result = scoring_to_dict(result_file, eval_f)
        model_name = result_file.name.rstrip(".json")
        if set_mode:
            model_name = model_name.rstrip("_intersection").rstrip("_union")
        model_name = (
            " ".join(model_name.rstrip(".json").split("-")[:2])
            if "openai" not in model_name
            else " ".join(model_name.rstrip(".json").split("-")[:3])
        )
        # capitalize the first letter of the name
        model_name = " ".join(
            [
                word.capitalize() if i == 0 else word
                for i, word in enumerate(model_name.split(" "))
            ]
        )
        model_name = re.sub(r"(\d)b", "\\1B", model_name)
        splits = model_name.split(" ")
        if len(splits) > 2:
            model_name = splits[0] + " " + "-".join(splits[1:])
        low_tokens = "low-tokens" in result_file.name
        entity_labels = "entity-labels" in result_file.name
        result_dict = {
            "Model": model_name,
            "LoRA": "\checkmark" if "lora" in result_file.name else "$\\times$",
            "RAG": "\checkmark" if "rag" in result_file.name else "$\\times$",
            "Reorder": "\checkmark" if "reorder" in result_file.name else "$\\times$",
            # "Low Tokens": "\checkmark"
            # if "low-tokens" in result_file.name
            # else "$\\times$",
            # "Entity Labels": "\checkmark"
            # if "entity-labels" in result_file.name
            # else "$\\times$",
        }
        if further_mode:
            result_dict["Low Tokens"] = "\checkmark" if low_tokens else "$\\times$"
            result_dict["Entity Labels"] = (
                "\checkmark" if entity_labels else "$\\times$"
            )
        if set_mode:
            result_dict["Set"] = set_op
        result_dict.update(eval_result)
        # result_dict.update({f"6_2_2_{k}": v for k, v in ternary_tag_score.items()})
        # result_dict.update({f"6_2_3_{k}": v for k, v in ternary_mention_score.items()})
        eval_results.append(result_dict)
    eval_df = pd.DataFrame(eval_results)
    if further_mode:
        eval_df = eval_df[
            (eval_df["Low Tokens"] == "\checkmark")
            | (eval_df["Entity Labels"] == "\checkmark")
        ]
    if merge_mode:
        eval_df = eval_df[eval_df["Model"].str.contains("Hermes 3B")]
    valid_cols = [
        c
        for c in [
            "Set",
            "Model",
            "RAG",
            "LoRA",
            "Reorder",
            "Low Tokens",
            "Entity Labels",
        ]
        if c in eval_df.columns
    ]
    eval_df.set_index(valid_cols, inplace=True)
    eval_df = eval_df.sort_index()
    # if "$F_{1,micro}$" in eval_df.columns:
    #     eval_df = eval_df.sort_values("$F_{1,micro}$")
    return eval_df


task_6_1_1_df = scoring_to_df(eval_submission_6_1_NER, res_dir=ner_result_dir)
task_6_2_1_df = scoring_to_df(eval_submission_6_2_binary_tag_RE)
task_6_2_2_df = scoring_to_df(eval_submission_6_3_ternary_tag_RE)
task_6_2_3_df = scoring_to_df(eval_submission_6_4_ternary_mention_RE)

Further mode: True
=== Removed 458 duplicated entities from predictions ===
=== Removed 111 overlapping entities ===
=== Removed 335 duplicated entities from predictions ===
=== Removed 86 overlapping entities ===
=== Removed 89 duplicated entities from predictions ===
=== Removed 35 overlapping entities ===
=== Removed 458 duplicated entities from predictions ===
=== Removed 111 overlapping entities ===
=== Removed 399 duplicated entities from predictions ===
=== Removed 61 overlapping entities ===
=== Removed 231 duplicated entities from predictions ===
=== Removed 82 overlapping entities ===
=== Removed 101 duplicated entities from predictions ===
=== Removed 48 overlapping entities ===
=== Removed 85 duplicated entities from predictions ===
=== Removed 64 overlapping entities ===
=== Removed 231 duplicated entities from predictions ===
=== Removed 82 overlapping entities ===
=== Removed 92 duplicated entities from predictions ===
=== Removed 61 overlapping entities ===
=== Removed 

<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:59: SyntaxWarning: invalid escape sequence '\c'
<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:61: SyntaxWarning: invalid escape sequence '\c'
<>:70: SyntaxWarning: invalid escape sequence '\c'
<>:72: SyntaxWarning: invalid escape sequence '\c'
<>:83: SyntaxWarning: invalid escape sequence '\c'
<>:84: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:34: SyntaxWarning: invalid escape sequence '\c'
<>:59: SyntaxWarning: invalid escape sequence '\c'
<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:61: SyntaxWarning: invalid escape sequence '\c'
<>:70: SyntaxWarning: invalid escape sequence '\c'
<>:72: SyntaxWarning: invalid escape sequence '\c'
<>:83: SyntaxWarning: invalid escape sequence '\c'
<>:84: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_225504/3103283138.py:34: SyntaxWarning: invalid escape sequence '\c

In [290]:
task_6_1_1_df[top_results:].to_latex(
    report_dir / "task_6_1_1.tex",
    float_format="%.2f",
    caption="Dev Set Result for Task 6.1.1 (NER) for various models and approaches.",
    label=f"tab:task:6_1_1{lbl_xtra}",
)
task_6_1_1_df

$P$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.048768   
                                         \checkmark $\times$       0.015713   
                                                    \checkmark     0.045376   
                              \checkmark $\times$   \checkmark     0.048768   
                                         \checkmark \checkmark     0.045376   
          \checkmark $\times$ $\times$   $\times$   \checkmark     0.212756   
                                         \checkmark $\times$       0.318396   
                                                    \checkmark     0.252185   
                              \checkmark $\times$   \checkmark     0.212756   
                                         \checkmark \checkmark     0.252185   

                                                                        $R$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.018770   
                                         \checkmark $\times$       0.017106   
                                                    \checkmark     0.016660   
                              \checkmark $\times$   \checkmark     0.018770   
                                         \checkmark \checkmark     0.016660   
          \checkmark $\times$ $\times$   $\times$   \checkmark     0.081851   
                                         \checkmark $\times$       0.066426   
                                                    \checkmark     0.054985   
                              \checkmark $\times$   \checkmark     0.081851   
                                         \checkmark \checkmark     0.054985   

                                                                      $F_1$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.025532   
                                         \checkmark $\times$       0.013724   
                                                    \checkmark     0.022781   
                              \checkmark $\times$   \checkmark     0.025532   
                                         \checkmark \checkmark     0.022781   
          \checkmark $\times$ $\times$   $\times$   \checkmark     0.112518   
                                         \checkmark $\times$       0.099484   
                                                    \checkmark     0.083981   
                              \checkmark $\times$   \checkmark     0.112518   
                                         \checkmark \checkmark     0.083981   

                                                                   $P_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.055046   
                                         \checkmark $\times$          0.037415   
                                                    \checkmark        0.057221   
                              \checkmark $\times$   \checkmark        0.055046   
                                         \checkmark \checkmark        0.057221   
          \checkmark $\times$ $\times$   $\times$   \checkmark        0.250000   
                                         \checkmark $\times$          0.292500   
                                                    \checkmark        0.308970   
                              \checkmark $\times$   \checkmark        0.250000   
                                         \checkmark \checkmark        0.308970   

                                                                   $R_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.0214

In [287]:
task_6_2_1_df[top_results:].to_latex(
    report_dir / "task_6_2_1.tex",
    float_format="%.2f",
    caption="Dev Set Result for Task 6.2.1 for various models and approaches.",
    label=f"tab:task:6_2_1{lbl_xtra}",
)
task_6_2_1_df

$P$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.064286   
                                         \checkmark $\times$       0.044444   
                                                    \checkmark     0.066049   
                              \checkmark $\times$   \checkmark     0.064286   
                                         \checkmark \checkmark     0.066049   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.240278   
                                                    \checkmark     0.158418   
                              \checkmark \checkmark \checkmark     0.158418   

                                                                        $R$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.086162   
                                         \checkmark $\times$       0.010582   
                                                    \checkmark     0.069638   
                              \checkmark $\times$   \checkmark     0.086162   
                                         \checkmark \checkmark     0.069638   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.091983   
                                                    \checkmark     0.072784   
                              \checkmark \checkmark \checkmark     0.072784   

                                                                      $F_1$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.057367   
                                         \checkmark $\times$       0.016667   
                                                    \checkmark     0.047191   
                              \checkmark $\times$   \checkmark     0.057367   
                                         \checkmark \checkmark     0.047191   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.124187   
                                                    \checkmark     0.088042   
                              \checkmark \checkmark \checkmark     0.088042   

                                                                   $P_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.153846   
                                         \checkmark $\times$          0.100000   
                                                    \checkmark        0.138614   
                              \checkmark $\times$   \checkmark        0.153846   
                                         \checkmark \checkmark        0.138614   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.654545   
                                                    \checkmark        0.465753   
                              \checkmark \checkmark \checkmark        0.465753   

                                                                   $R_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.081818   
                                         \checkmark $\times$          0.009091   
                                                    \checkmark        0.063636   
                              \checkmark $\times$   \checkmark        0.081818   
                                         \checkmark \checkmark        0.063636   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.163636   
                                                    \checkmark        0.154545   
                              \checkmark \checkmark \checkmark        0.154545   

                                                         

In [288]:
task_6_2_2_df[top_results:].to_latex(
    report_dir / "task_6_2_2.tex",
    float_format="%.2f",
    caption="Further Dev Set Result for Task 6.2.2 for various models and approaches.",
    label=f"tab:task:6_2_2{lbl_xtra}",
)
task_6_2_2_df

$P$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.073313   
                                         \checkmark $\times$       0.041667   
                                                    \checkmark     0.074074   
                              \checkmark $\times$   \checkmark     0.073313   
                                         \checkmark \checkmark     0.074074   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.214844   
                                                    \checkmark     0.148516   
                              \checkmark \checkmark \checkmark     0.148516   

                                                                        $R$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.084183   
                                         \checkmark $\times$       0.009921   
                                                    \checkmark     0.067895   
                              \checkmark $\times$   \checkmark     0.084183   
                                         \checkmark \checkmark     0.067895   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.082479   
                                                    \checkmark     0.069416   
                              \checkmark \checkmark \checkmark     0.069416   

                                                                      $F_1$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.057211   
                                         \checkmark $\times$       0.015625   
                                                    \checkmark     0.046027   
                              \checkmark $\times$   \checkmark     0.057211   
                                         \checkmark \checkmark     0.046027   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.112799   
                                                    \checkmark     0.084230   
                              \checkmark \checkmark \checkmark     0.084230   

                                                                   $P_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.140351   
                                         \checkmark $\times$          0.105263   
                                                    \checkmark        0.120000   
                              \checkmark $\times$   \checkmark        0.140351   
                                         \checkmark \checkmark        0.120000   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.636364   
                                                    \checkmark        0.465753   
                              \checkmark \checkmark \checkmark        0.465753   

                                                                   $R_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.069565   
                                         \checkmark $\times$          0.008696   
                                                    \checkmark        0.052174   
                              \checkmark $\times$   \checkmark        0.069565   
                                         \checkmark \checkmark        0.052174   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.152174   
                                                    \checkmark        0.147826   
                              \checkmark \checkmark \checkmark        0.147826   

                                                         

In [289]:
task_6_2_3_df[top_results:].to_latex(
    report_dir / "task_6_2_3.tex",
    float_format="%.2f",
    caption="Dev Set Result for Task 6.2.3 for various models and approaches.",
    label=f"tab:task:6_2_3{lbl_xtra}",
)
task_6_2_3_df

$P$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.000631   
                                         \checkmark $\times$       0.000000   
                                                    \checkmark     0.001736   
                              \checkmark $\times$   \checkmark     0.000631   
                                         \checkmark \checkmark     0.001736   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.028837   
                                                    \checkmark     0.031675   
                              \checkmark \checkmark \checkmark     0.031675   

                                                                        $R$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.003472   
                                         \checkmark $\times$       0.000000   
                                                    \checkmark     0.003472   
                              \checkmark $\times$   \checkmark     0.003472   
                                         \checkmark \checkmark     0.003472   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.008060   
                                                    \checkmark     0.011175   
                              \checkmark \checkmark \checkmark     0.011175   

                                                                      $F_1$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels             
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark     0.001068   
                                         \checkmark $\times$       0.000000   
                                                    \checkmark     0.002315   
                              \checkmark $\times$   \checkmark     0.001068   
                                         \checkmark \checkmark     0.002315   
          \checkmark $\times$ $\times$   \checkmark $\times$       0.010926   
                                                    \checkmark     0.015035   
                              \checkmark \checkmark \checkmark     0.015035   

                                                                   $P_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.002924   
                                         \checkmark $\times$          0.000000   
                                                    \checkmark        0.004525   
                              \checkmark $\times$   \checkmark        0.002924   
                                         \checkmark \checkmark        0.004525   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.062802   
                                                    \checkmark        0.084071   
                              \checkmark \checkmark \checkmark        0.084071   

                                                                   $R_{micro}$  \
Model     RAG        LoRA     Reorder    Low Tokens Entity Labels                
Hermes 3B $\times$   $\times$ $\times$   $\times$   \checkmark        0.001786   
                                         \checkmark $\times$          0.000000   
                                                    \checkmark        0.001786   
                              \checkmark $\times$   \checkmark        0.001786   
                                         \checkmark \checkmark        0.001786   
          \checkmark $\times$ $\times$   \checkmark $\times$          0.023214   
                                                    \checkmark        0.033929   
                              \checkmark \checkmark \checkmark        0.033929   

                                                         